# Idee der Householdertransformation

In [57]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive
from numpy.linalg import norm

In [58]:
np.set_printoptions(precision=2,suppress=True)

In [59]:
def HouseholderTransformation(w):
    return np.eye(w.shape[0])-2*np.outer(w,w)/np.dot(w,w)

Gesucht ist daher der geeignete Normalenvektor so, dass der gespiegelte Spaltenvektor auf die $e_1$ Achse zu liegen kommt.

Sei mit $y$ der Spaltenvektor bezeichnet, so kann man zeigen (siehe Skript), dass der Vektor

$$w = y \pm \|y\|_2 e_1$$

die gewünschte Eigenschaft hat. Um **Auslöschung** in der Berechnung von $w$ zu vermeiden, wählt man

$$w = y + \text{sign}(y_1) \|y\|_2 e_1$$

mit

$$\text{sign}(s) = \begin{cases} 1 & \quad \text{für} s \ge 0\\ -1 & \quad \text{sonst}.\end{cases}$$



In [60]:
def mysign(x): # numpy sign liefert 0 für 0
    if x >= 0:
        return 1
    else:
        return -1

Funktion für den n-dimensionalen Einheitsvektor

In [61]:
def e(n):
    return np.array([1]+[0 for k in range(n-1)])

## Schrittweise QR-Zerlegung mit Hilfe der Householder-Transformation

Mit Hilfe der Householder-Transformation soll nun die Matrix $A$ in eine orthogonale Matrix $Q$ und reguläre obere Dreiecksmatrix $R$ zerlegt werden.

Im Beispiel wählen wir eine beliebige Matrix $A \in \mathbb{R}^{5\times5}$.

In [62]:
A = np.array([[-1,  7, -8, -9,  6],
       [-6, -8,  0,  3,  8],
       [-4, -2,  8,  0, -2],
       [-1, -9,  4, -8,  2],
       [-3, -5, -5,  7, -4]
       ],dtype=float)
m,n = A.shape
print(m,n)

5 5


### 1. Spalte

In [63]:
k = 0

Die Hyperebene ist definiert durch

In [64]:
w = A[k:,k] + mysign(A[k,k])*norm(A[k:,k])*e(m-k)
print(w)

[-8.94 -6.   -4.   -1.   -3.  ]


Für die Householder-Transformationsmatrix angewand auf $A$ erhalten wir

In [65]:
Q1 = HouseholderTransformation(w)
print('Q1:\n', Q1)
A1 = Q1@A
print('A1:\n', A1)

Q1:
 [[-0.13 -0.76 -0.5  -0.13 -0.38]
 [-0.76  0.49 -0.34 -0.08 -0.25]
 [-0.5  -0.34  0.77 -0.06 -0.17]
 [-0.13 -0.08 -0.06  0.99 -0.04]
 [-0.38 -0.25 -0.17 -0.04  0.87]]
A1:
 [[ 7.94  9.2  -1.64 -2.77 -4.54]
 [-0.   -6.52  4.27  7.18  0.93]
 [ 0.   -1.02 10.85  2.79 -6.72]
 [ 0.   -8.75  4.71 -7.3   0.82]
 [-0.   -4.26 -2.86  9.09 -7.54]]


In der ersten Spalte der Zwischenmatrix $A_1$ stehen nun abgesehen vom ersten Eintrag Nullen:

In [66]:
print(A1)

[[ 7.94  9.2  -1.64 -2.77 -4.54]
 [-0.   -6.52  4.27  7.18  0.93]
 [ 0.   -1.02 10.85  2.79 -6.72]
 [ 0.   -8.75  4.71 -7.3   0.82]
 [-0.   -4.26 -2.86  9.09 -7.54]]


### 2. Spalte 

In [67]:
k = 1

Die Hyperebene ist definiert durch

In [68]:
w = A1[k:,k] + mysign(A1[k,k])*norm(A1[k:,k])*e(m-k)
print(w)

[-18.29  -1.02  -8.75  -4.26]


wobei nun das letzte Resultat $A_1$ benutzt wird. Die Householder-Transformationsmatrix wird nun nur auf die Submatrix von $A_1$ angewand und in der Submatrix von $A_1$ wiederum gespeichert:

In [69]:
Q2 = HouseholderTransformation(w)
A1[k:,k:] = Q2@A1[k:,k:]


Die Dimension der zweiten Householder-Transformationsmatrix $Q_2$ ist

In [70]:
Q2.shape

(4, 4)

In dem ersten beiden Spalte der Zwischenmatrix $A_1$ steht:

In [71]:
print(A1)

[[  7.94   9.2   -1.64  -2.77  -4.54]
 [ -0.    11.76  -5.77  -2.08   2.19]
 [  0.     0.    10.29   2.27  -6.65]
 [  0.    -0.    -0.1  -11.74   1.42]
 [ -0.    -0.    -5.21   6.93  -7.24]]


### 3. - 5. Spalte 

Wir automatisieren nun den Prozess und überschreiben die Submatrizen der Matrix $A_1$ sukzessive:

In [72]:
for k in range(2,n):
    print('Spalte '+str(k+1))
    w = A1[k:,k] + mysign(A1[k,k])*norm(A1[k:,k])*e(m-k)
    Qk = HouseholderTransformation(w)
    A1[k:,k:] = Qk@A1[k:,k:]
    print(A1)

Spalte 3
[[  7.94   9.2   -1.64  -2.77  -4.54]
 [ -0.    11.76  -5.77  -2.08   2.19]
 [  0.     0.   -11.53   1.     2.67]
 [  0.    -0.     0.   -11.73   1.38]
 [ -0.    -0.     0.     7.23  -9.47]]
Spalte 4
[[  7.94   9.2   -1.64  -2.77  -4.54]
 [ -0.    11.76  -5.77  -2.08   2.19]
 [  0.     0.   -11.53   1.     2.67]
 [  0.    -0.     0.    13.78  -6.15]
 [ -0.    -0.     0.    -0.    -7.33]]
Spalte 5
[[  7.94   9.2   -1.64  -2.77  -4.54]
 [ -0.    11.76  -5.77  -2.08   2.19]
 [  0.     0.   -11.53   1.     2.67]
 [  0.    -0.     0.    13.78  -6.15]
 [ -0.    -0.     0.    -0.     7.33]]
